In [ ]:
from cv2 import cv2
import numpy as np

In [ ]:
image = cv2.imread("cat.png",cv2.IMREAD_GRAYSCALE)

In [ ]:
#利用Python语言环境编写基于形态学方法的图像边缘特征检测程序
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
dilate_img = cv2.dilate(image,kernel)
erode_img = cv2.erode(image,kernel)

In [ ]:
absdiff_img = cv2.absdiff(dilate_img,erode_img)
retval,threshold_img = cv2.threshold(absdiff_img,40,255,cv2.THRESH_BINARY)
result = cv2.bitwise_not(threshold_img)

In [ ]:
#显示图像
cv2.imshow("original_image",image)
cv2.imshow("dilate_img",dilate_img)
cv2.imshow("eroce_img",erode_img)
cv2.imshow("absdiff_img",absdiff_img)
cv2.imshow("threshold_img",threshold_img)
cv2.imshow("result",result)

In [ ]:
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#基于开运算和闭运算的图像去噪程序
original_img = image
gray_res = cv2.resize(original_img,None,fx=0.8,fy=0.8,
                     interpolation = cv2.INTER_CUBIC)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
#闭运算1
closed1 = cv2.morphologyEx(gray_res,cv2.MORPH_CLOSE,kernel,iterations=1)
#闭运算2
closed2 = cv2.morphologyEx(gray_res,cv2.MORPH_CLOSE,kernel,iterations=3)
#开运算1
opened1 = cv2.morphologyEx(gray_res,cv2.MORPH_OPEN,kernel,iterations=1)
#开运算2
opened2 = cv2.morphologyEx(gray_res,cv2.MORPH_OPEN,kernel,iterations=3)
#梯度
gradient = cv2.morphologyEx(gray_res,cv2.MORPH_GRADIENT,kernel)

In [ ]:
#显示图像
cv2.imshow("gray_res",gray_res)
cv2.imshow("Close1",closed1)
cv2.imshow("Close2",closed2)
cv2.imshow("Open1",opened1)
cv2.imshow("Open2",opened2)
cv2.imshow("gradient",gradient)

In [ ]:
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#车牌与背景分割
img = cv2.imread("car2.jpg")
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.imshow('gray',img_gray)
cv2.waitKey(0)

In [ ]:
#将灰度图图像二值化，设定阈值是100
img_thre = img_gray
cv2.threshold(img_gray,100,255,cv2.THRESH_BINARY_INV,img_thre)
cv2.imshow("threshold",img_thre)
cv2.waitKey(0)

In [ ]:
#保存黑白图片
cv2.imwrite("thre_res.png",img_thre)

In [ ]:
#分割字符
white = []
black = []
height = img_thre.shape[0]
width = img_thre.shape[1]
white_max = 0
black_max = 0
#计算每一列的黑白色像素总和
for i in range(width):
    s = 0
    t = 0
    for j in range(height):
        if img_thre[j][i] == 255:
            s += 1
        if img_thre[j][i] == 0:
            t += 1
    white_max = max(white_max,s)
    black_max = max(black_max,t)
    white.append(s)
    black.append(t)
    print(s)
    print(t)

arg = False
if black_max > white_max:
    arg = True
    
#分割图像
def find_end(start_):
    end_ = start_ + 1
    for m in range(start_ + 1 ,width - 1):
        if (black[m] if arg else white[m]) > (0.625 * black_max if arg else 0.625 * white_max):
            end_ = m
            break
    return end_

In [ ]:
n = 1
start = 1
end = 2
while n < width-2:
    n += 1 
    if (white[n] if arg else black[n])>(0.375 * white_max if arg else 0.375 * black_max):
        start = n
        end = find_end(start)
        n = end
        if end - start > 5:
            cj = img_thre[1:height,start:end]
            cv2.imshow("canjian",cj)
            cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
from PIL import Image
import os.path
from skimage import io,data
from cv2 import cv2
import numpy as np

In [5]:
def stretch(img):
    maxi = float(img.max())
    mini = float(img.min())
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            img[i,j] = (255/(maxi-mini)*img[i,j]-(255*mini)/(maxi-mini))
    return img

def dobinaryzation(img):
    maxi = float(img.max())
    mini = float(img.min())
    x = maxi - ((maxi-mini)/2)
    ret,thresh = cv2.threshold(img,x,255,cv2.THRESH_BINARY)
    return thresh

def find_rectangle(contour):
    y,x = [],[]
    for p in contour:
        y.append(p[0][0])
        x.append(p[0][1])
    return [min(y),min(x),max(y),max(x)]

def locate_license(img,afterimg):
    img,contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    block = []
    for c in contours:
        r = find_rectangle(c)
        a = (r[2]-r[0])*(r[3]-r[1])
        s = (r[2]-r[0])*(r[3]-r[1])
        block.append([r,a,s])
    block = sorted(block,key=lambda b:b[1])[-3:]
    maxweight,maxindex = 0,-1
    for i in range(len(block)):
        b = afterimg[block[i][0][1]:block[i][0][3],block[i][0][0]:block[i][0][2]]
        hsv = cv2.cvtColor(b,cv2.COLOR_BGR2HSV)
        lower = np.array([100,50,50])
        upper = np.array([140,255,255])
        mask = cv2.inRange(hsv,lower,upper)
        w1 = 0
        for m in mask:
            w1 += m/255
        w2 = 0
        for n in w1:
            w2 += n
        if w2 > maxweight:
            maxindex = i
            maxweight = w2
    return block[maxindex][0]

def find_license(img):
    m = 400 * img.shape[0]/img.shape[1]
    img = cv2.resize(img,(400,int(m)),interpolation=cv2.INTER_CUBIC)
    gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    stretchedimg = stretch(gray_img)
    r = 16
    h = w = r * 2 + 1
    kernel = np.zeros((h,w),np.uint8)
    cv2.circle(kernel,(r,r),r,1,-1)
    openingimg = cv2.morphologyEx(stretchedimg,cv2.MORPH_OPEN,kernel)
    strtimg = cv2.absdiff(stretchedimg,openingimg)
    binaryimg = dobinaryzation(strtimg)
    canny = cv2.Canny(binaryimg,binaryimg.shape[0],binaryimg.shape[1])
    kernel = np.ones((5,19),np.uint8)
    closingimg = cv2.morphologyEx(canny,cv2.MORPH_CLOSE,kernel)
    openingimg = cv2.morphologyEx(closingimg,cv2.MORPH_OPEN,kernel)
    rect = locate_license(openingimg,img)
    return rect,img

def cut_lincense(afterimg,rect):
    rect[2] = rect[2] - rect[0]
    rect[3] = rect[3] - rect[1]
    rect_copy = tuple(rect.copy())
    rect = [0,0,0,0]
    mask = np.zeros(afterimg.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    cv2.grabCut(afterimg,mask,rect_copy,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('unit8')
    img_show = afterimg*mask2[:,:,np.newaxis]
    return img_show

def deal_lincense(lincenseimg):
    gray_img = cv2.cvtColor(lincenseimg,cv2.COLOR_BGR2GRAY)
    kernel = np.ones((3,3),np.float32)/9
    gray_img = cv2.filter2D(gray_img,-1,kernel)
    ret,thresh = cv2.threshold(gray_img,120,255,cv2.THRESH_BINARY)
    return thresh

def find_end(start,arg,black,white,width,black_max,white_max):
    end = start + 1
    for m in range(start+1,width-1):
        if(black[m] if arg else white[m]) > (0.98 * black_max if arg else 0.98 * white_max):
            end = m
            break
    return end

In [7]:
img = cv2.imread("pure.jpeg",cv2.IMREAD_COLOR)
rect,afterimg = find_license(img)
cv2.rectangle(afterimg,(rect[0],rect[1]),(rect[2],rect[3]),(0,255,0),2)
cv2.imshow("afterimg",afterimg)
cutimg = cut_lincense(afterimg,rect)
cv2.imshow("cutimg",cutimg)
thresh = deal_lincense(cutimg)
cv2.imshow("thresh",thresh)
cv2.waitKey(0)

white = []
black = []
height = thresh.shape[0]
width = thresh.shape[1]
white_max = 0
black_max = 0
for i in range(width):
    line_white = 0
    line_black = 0
    for j in range(height):
        if thresh[j][i] == 255:
            line_white += 1
        if thresh[j][i] == 0:
            line_black += 1
    white_max = max(white_max,line_white)
    black_max = max(black_max,line_black)
    white.append(line_white)
    black.append(line_black)
    print("white",white)
    print("black",black)

arg = True
if black_max < white_max :
    arg = False

n = 1
start = 1
end = 2
s_width = 28
s_height = 28
while n < width - 2 :
    n += 1
    if(white[n] if arg else black[n]) > (0.02 * white_max if arg else 0.02 * black_max):
        start = n
        end = find_end(start,arg,black,white,black_max,white_max)
        n = end
    if end - start > 5:
        cj = thresh[1:height,start:end]
        print("result/%s.jpg"%(n))
        infilte = "result/%s.jpg"%(n)
        io.imsave(infilte,cj)
        cv2.imshow("cutlincense",cj)
        cv2.waitKey(0)

cv2.waitKey(0)
cv2.destroyAllWindows()



ValueError: not enough values to unpack (expected 3, got 2)